# GASS-PPI

## Section 0: Libraries, Global Variables, Utility Functions

In [23]:
import subprocess
import re
import numpy as np
from Bio.PDB import *

repository_path = "/Users/albertdarmawan/Documents/gass-ppi/"
dbd5_path = repository_path + "dataset/benchmark5/structures/"
pdb_parser = PDBParser()

# List of PDB ID of protein complexes
# aa stands for Antibody-Antigen (in this case, Antibody is the receptor and Antigen is the ligand)
dbd5_aa_list = ["1AHW", "1BVK", "1DQJ", "1E6J", "1JPS", "1MLC", "1VFB", "1WEJ",
                              "2FD6", "2I25", "2VIS", "2VXT", "2W9E", "3EOA", "3HMX", "3MXW",
                              "3RVW", "4DN4", "4FQI", "4G6J", "4G6M", "4GXU", "3EO1", "3G6D",
                              "3HI6", "3L5W", "3V6Z", "1BGX"]

dbd5_aa_r_list = list(map(lambda x: x + "_r_u", dbd5_aa_list))
dbd5_aa_l_list = list(map(lambda x: x + "_l_u", dbd5_aa_list))
# example = pdb_parser.get_structure("1AHW_L", dataset_directory + "1AHW_l_u.pdb")
lha_dict = {
    "GLY":"CA",
    "ALA":"CB",
    "GLN":"CD",
    "GLU":"CD",
    "ILE":"CD1",
    "LEU":"CD1",
    "MET":"CE",
    "HIS":"CE1",
    "ASN":"CG",
    "ASP":"CG",
    "PRO":"CG",
    "VAL":"CG1",
    "THR":"CG2",
    "TRP":"CH2",
    "ARG":"CZ",
    "PHE":"CZ",
    "LYS":"NZ",
    "SER":"OG",
    "TYR":"OH",
    "CYS":"SG",
}

In [24]:
# In this case, a Residue is represented by a reference atom (LHA, CA, etc)
# Residue is the "gene" in genetic algorithms.
# Therefore, a list of Residue corresponds to an "individual"
# A residue consists of 2 residue information, 1 chain information, and 2 reference atom information
class Residue:
    def __init__(self, residue_name, residue_sequence_position, chain_name, atom_name, atom_coordinates):
        self.residue_name = residue_name
        self.residue_sequence_position = residue_sequence_position
        self.chain_name = chain_name
        self.atom_name = atom_name
        self.atom_coordinates = atom_coordinates

In [25]:
def euclidean_distance(coordinate_1, coordinate_2):
    """Euclidean Distance
    Given 3-dimensional coordinates of 2 atoms, calculate its Euclidean distance
    
    Parameters:
    coordinate_1 (1D NumPy Array): x,y,z coordinates of the first atom
    coordinate_2 (1D NumPy Array): x,y,z coordinates of the second atom
    
    Returns:
    float: The euclidean distance
    
    """
    return float(np.sqrt(((coordinate_1[0] - coordinate_2[0]) ** 2) +
                   ((coordinate_1[1] - coordinate_2[1]) ** 2) +
                   ((coordinate_1[2] - coordinate_2[2]) ** 2)))

In [26]:
def load_pdb(pdb_id, pdb_directory_path, pdb_parser, lha_dict, reference_atom="lha"):
    """Load PDB
    Given a PDB ID, load the .pdb file and retrieve the list of residue
    
    Parameters:
    pdb_id (str): The PDB ID for the protein structure
    reference_atom (str): Reference atom used ("lha" or "ac")
    
    Returns:
    list: List of Residue object which constitutes the protein structure
    
    """
    residue_list = []
    amino_acid_list = list(lha_dict.keys())
    pdb_file_path = pdb_directory_path + pdb_id + ".pdb"
    pdb_structure = pdb_parser.get_structure(pdb_id, pdb_file_path)
    # Only take the ATOM keyword (exclude the HETATM, hetero atom that is not inside standard amino acids)
    biopdb_residue_list = [residue for residue in pdb_structure.get_residues() if residue.get_resname() in amino_acid_list]
    if reference_atom == "lha":
        biopdb_atom_list = [atom for residue in biopdb_residue_list for atom in residue if atom.get_name() == lha_dict[residue.get_resname()]]
    else:
        biopdb_atom_list = [atom for residue in biopdb_residue_list for atom in residue if atom.get_name() == "CA"]
        
    for atom in biopdb_atom_list:
        # Create a new Residue instance
        current_residue_name = atom.get_parent().get_resname()
        current_residue_sequence_position = atom.get_parent().get_full_id()[3][1]
        current_chain_name = atom.get_parent().get_parent().id
        current_atom_name = atom.get_name()
        current_atom_coordinates = atom.get_coord()
        current_atom = Residue(current_residue_name,
                               current_residue_sequence_position,
                               current_chain_name,
                               current_atom_name,
                               current_atom_coordinates)
        residue_list.append(current_atom)
    return residue_list

residue_3nos = load_pdb("1AHW_l_u", dbd5_path, pdb_parser, lha_dict, "lha")
print(type(residue_3nos))
print(len(residue_3nos))
for item in residue_3nos[:5]:
    print(item.residue_name)
    print(item.residue_sequence_position)
    print(item.chain_name)
    print(item.atom_name)
    print(item.atom_coordinates)

<class 'list'>
202
ASN
5
A
CG
[-34.951 -48.179  13.288]
THR
6
A
CG2
[-30.526 -48.949  17.813]
VAL
7
A
CG1
[-30.492 -41.64   18.021]
ALA
8
A
CB
[-24.799 -41.51   17.044]
ALA
9
A
CB
[-25.297 -39.985  22.243]


/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1623
  warnings.warn(


In [27]:
def tmalign_structural_alignment(pdb_id_1, pdb_id_2, pdb_directory_path):
    """TMAlign Structural Alignment
    Compare structural similarities between two PDB structures (regardless of the rotation)
    https://zhanggroup.org/TM-score/
    
    Parameters:
    pdb_id_1 (str): The PDB ID for the first protein structure
    pdb_id_2 (str): The PDB ID for the second protein structure
    
    Returns:
    float: The TMScore, a value between (0,1]. 1 indicates a perfect match. >0.5 is similar enough. <0.17 is two unrelated structures.
    
    """
    pdb_id_1_file_path = pdb_directory_path + pdb_id_1 + ".pdb"
    pdb_id_2_file_path = pdb_directory_path + pdb_id_2 + ".pdb"
    # Execute TMAlign
    tmalign_thread = subprocess.run(["./TMalign", pdb_id_1_file_path, pdb_id_2_file_path], capture_output=True, text=True)
    output_text = tmalign_thread.stdout

    # Retrieved TMScore from TMAlign results
    tmscore_raw_list = re.findall("TM-score=\s[0-9]+.[0-9]+", output_text)

    # Convert the TMScore into floats, then get the maximum TMScore
    tmscore_list = list(map(lambda x: float(re.sub("TM-score=\s", "", x)), tmscore_raw_list))
    max_tmscore = max(tmscore_list)
    return max_tmscore

tmscore = tmalign_structural_alignment("1AHW_l_u", "1BVK_l_u", dbd5_path)
print(tmscore)

0.23652


In [28]:
# Given one receptor and one ligand, infer its interfaces
# The protein-protein interfaces is a list of Residue class
def get_actual_interface(residue_list_1, residue_list_2, threshold=6.0):
    """Get Protein Interface
    Given one receptor structure and one ligand structure, infer its interfaces
    
    Parameters:
    residue_list_1 (list): List of Residue object from the first protein structure
    residue_list_2 (list): List of Residue object from the second protein structure
    threshold: The acceptable distance between a receptor's atom and a ligand's atom
    
    Returns:
    list: List of Residue object which constitutes the protein interface
    
    """
    interface_list = []
    for residue_1 in residue_list_1:
        for residue_2 in residue_list_2:
            current_distance = euclidean_distance(residue_1.atom_coordinates, residue_2.atom_coordinates)
            if current_distance < threshold:
                interface_list.append(residue_1)
                break

    return interface_list

ligand = load_pdb("1AHW_l_u", dbd5_path, pdb_parser, lha_dict, "lha")
receptor = load_pdb("1AHW_r_u", dbd5_path, pdb_parser, lha_dict, "lha")
sample_interface_list = get_actual_interface(ligand, receptor)
print(sample_interface_list)
print(type(sample_interface_list))
print(len(sample_interface_list))
for item in sample_interface_list:
    print(item.residue_name)

/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1623
  warnings.warn(
/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3305
  warnings.warn(


[<__main__.Residue object at 0x7fe0cf4e5340>, <__main__.Residue object at 0x7fe0cf4e5d60>, <__main__.Residue object at 0x7fe0cf4e5dc0>, <__main__.Residue object at 0x7fe0cf4e5e80>, <__main__.Residue object at 0x7fe0cf4e5f40>, <__main__.Residue object at 0x7fe0cf4e8040>, <__main__.Residue object at 0x7fe0cf4e80a0>, <__main__.Residue object at 0x7fe0cf4e8400>, <__main__.Residue object at 0x7fe0cf4e8460>, <__main__.Residue object at 0x7fe0cf4e84c0>, <__main__.Residue object at 0x7fe0cf4e8520>, <__main__.Residue object at 0x7fe0cf4e8580>, <__main__.Residue object at 0x7fe0cf4e85e0>, <__main__.Residue object at 0x7fe0cf4e8640>, <__main__.Residue object at 0x7fe0cf4e8880>, <__main__.Residue object at 0x7fe0cf4e8d00>, <__main__.Residue object at 0x7fe0cf4e8dc0>, <__main__.Residue object at 0x7fe0cf4e8e80>, <__main__.Residue object at 0x7fe0cf4e8ee0>, <__main__.Residue object at 0x7fe0cf4e8fa0>, <__main__.Residue object at 0x7fe0cf5cd100>, <__main__.Residue object at 0x7fe0cf5cd160>, <__main__

In [29]:
def gass_ppi(residue_list, interface_template):
    return 0

In [30]:
def evaluate(predicted_interface, actual_interface):
    precision = 0.5
    recall = 0.6
    auc_roc_score = 0.7
    auc_pr_score = 0.8
    return (precision, recall, auc_roc_score, auc_pr_score)

## Section 1: Proof-of-Concept

In [32]:
# Query Protein
input_pdb_id = "1AHW_r_u"
input_pdb_structure = load_pdb(input_pdb_id, dbd5_path, pdb_parser, lha_dict, "lha")
# Remove the first element in this case, since it's the input_pdb_id
dataset_list = dbd5_aa_r_list[1:]

# Step 1: Find the structural neighbour of the query protein using TMAlign
tmscore_list = list(map(lambda x: tmalign_structural_alignment(input_pdb_id, x, dbd5_path), dataset_list))
maximum_index = np.argmax(tmscore_list)
neighbour_pdb_id = dataset_list[maximum_index]

# Step 2: Find the interface of the structural neighbour, then use it as the interface template
neighbour_pair_pdb_id = neighbour_pdb_id[:5] + "l" + neighbour_pdb_id[6:]

neighbour_pdb_structure = load_pdb(neighbour_pdb_id, dbd5_path, pdb_parser, lha_dict, "lha")
neighbour_pair_pdb_structure = load_pdb(neighbour_pair_pdb_id, dbd5_path, pdb_parser, lha_dict, "lha")

interface_template = get_actual_interface(neighbour_pdb_structure, neighbour_pair_pdb_structure)

# Step 3: GASS-PPI
predicted_interface = gass_ppi(input_pdb_structure, interface_template)

# Step 4: Evaluation
input_pair_pdb_id = input_pdb_id[:5] + "l" + input_pdb_id[6:]
input_pair_pdb_structure = load_pdb(input_pair_pdb_id, dbd5_path, pdb_parser, lha_dict, "lha")
actual_interface = get_actual_interface(input_pdb_structure, input_pair_pdb_structure)

precision, recall, auc_roc_score, auc_pr_score = evaluate(predicted_interface, actual_interface)
print("Precision: ", precision)
print("Recall: ", recall)
print("AUC-ROC Score: ", auc_roc_score)
print("AUC-PR Score: ", auc_pr_score)

/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3305
  warnings.warn(
/opt/anaconda3/envs/thesis/lib/python3.8/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 1623
  warnings.warn(


Precision:  0.5
Recall:  0.6
AUC-ROC Score:  0.7
AUC-PR Score:  0.8
